In [64]:
import pandas_datareader as web
import datetime
import pandas as pd
import yfinance as yf
from datetime import date

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt
from pypfopt.efficient_frontier import EfficientCVaR

In [65]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [66]:
start = '2010-01-01'
end = date.today()

stock = ['DIS', 'NFLX', 'PARA', 'CMCSA', 'ROKU', 'FOXA', 'DISCA', 'WMG', 'CURI', 'LYV', 'T']

In [67]:
def get_stock(stock):
    for x in stock:
        data = yf.download(stock, start, end)
        c_data = data['Close']
    return c_data

In [90]:
portfolio=get_stock(stock).dropna()
portfolio

[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  11 of 11 completed


,CMCSA,CURI,DIS,DISCA,FOXA,LYV,NFLX,PARA,ROKU,T,WMG
Date,,,,,,,,,,,
2020-06-03,41.299999,9.750000,122.180000,22.270000,30.770000,53.910000,421.970001,22.719999,103.959999,31.510000,30.120001
2020-06-04,41.290001,9.750000,123.690002,23.090000,30.530001,53.470001,414.329987,24.120001,102.930000,31.799999,30.000000
2020-06-05,42.090000,9.750000,124.820000,24.780001,30.850000,56.959999,419.600006,24.820000,104.379997,32.770000,29.940001
2020-06-08,43.119999,9.750000,127.279999,26.150000,32.480000,57.090000,419.489990,28.510000,112.949997,33.230000,31.000000
2020-06-09,42.150002,9.950000,123.889999,23.879999,30.809999,53.419998,434.049988,26.020000,113.250000,32.619999,32.000000
2020-06-10,41.360001,9.860000,122.180000,23.170000,30.139999,52.480000,434.480011,24.000000,113.800003,32.139999,31.049999
2020-06-11,39.250000,9.850000,112.639999,22.340000,28.059999,47.740002,425.559998,23.590000,106.949997,30.170000,29.990000
2020-06-12,39.459999,9.855000,115.489998,22.070000,28.290001,49.119999,418.070007,22.900000,108.070000,30.500000,30.770000
2020-06-15,39.830002,9.855000,117.080002,22.750000,29.290001,49.400002,425.500000,24.889999,107.489998,30.500000,32.090000


In [91]:
mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [92]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))


{'CMCSA': 0.0, 'CURI': 0.0, 'DIS': 0.0, 'DISCA': 0.0, 'FOXA': 0.07292, 'LYV': 0.88584, 'NFLX': 0.0, 'PARA': 0.0, 'ROKU': 0.03085, 'T': 0.0, 'WMG': 0.01038}


In [93]:
ef.portfolio_performance(verbose=True)

Expected annual return: 58.1%
Annual volatility: 39.8%
Sharpe Ratio: 1.41


(0.5807577762302316, 0.39841913915068167, 1.4074569244479835)

In [94]:
latest_prices = get_latest_prices(portfolio)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print('Discrete Allocation', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))

Discrete Allocation {'LYV': 702, 'FOXA': 175, 'ROKU': 22, 'WMG': 29}
Funds remaining: $79.96


In [95]:
'HRP'

returns = portfolio.pct_change().dropna()

In [96]:
returns

,CMCSA,CURI,DIS,DISCA,FOXA,LYV,NFLX,PARA,ROKU,T,WMG
Date,,,,,,,,,,,
2020-06-04,-0.000242,0.000000,0.012359,0.036821,-0.007800,-0.008162,-0.018106,0.061620,-0.009908,0.009203,-0.003984
2020-06-05,0.019375,0.000000,0.009136,0.073192,0.010481,0.065270,0.012719,0.029022,0.014087,0.030503,-0.002000
2020-06-08,0.024471,0.000000,0.019708,0.055286,0.052836,0.002282,-0.000262,0.148670,0.082104,0.014037,0.035404
2020-06-09,-0.022495,0.020513,-0.026634,-0.086807,-0.051416,-0.064284,0.034709,-0.087338,0.002656,-0.018357,0.032258
2020-06-10,-0.018743,-0.009045,-0.013803,-0.029732,-0.021746,-0.017596,0.000991,-0.077633,0.004857,-0.014715,-0.029688
2020-06-11,-0.051015,-0.001014,-0.078082,-0.035822,-0.069011,-0.090320,-0.020530,-0.017083,-0.060193,-0.061294,-0.034138
2020-06-12,0.005350,0.000508,0.025302,-0.012086,0.008197,0.028907,-0.017600,-0.029250,0.010472,0.010938,0.026009
2020-06-15,0.009377,0.000000,0.013767,0.030811,0.035348,0.005700,0.017772,0.086900,-0.005367,0.000000,0.042899
2020-06-16,-0.002009,0.004566,0.011616,-0.021538,-0.033459,-0.004049,0.024982,-0.028124,0.123919,0.009180,-0.003428


In [97]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()


In [98]:
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 12.5%
Annual volatility: 19.4%
Sharpe Ratio: 0.54
{'CMCSA': 0.18422169759391155, 'CURI': 0.033005921475757795, 'DIS': 0.12012351712503372, 'DISCA': 0.05575656245286289, 'FOXA': 0.0736138088038254, 'LYV': 0.07060485196600418, 'NFLX': 0.0710093950352131, 'PARA': 0.049202215642683456, 'ROKU': 0.027861961062029417, 'T': 0.19975797813543872, 'WMG': 0.11484209070723977}


In [99]:
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_hrp.greedy_portfolio()
print('Discrete allocation (HRP):', allocation)
print('Funds remaining (HRP): ${:.2f}'.format(leftover))

Discrete allocation (HRP): {'T': 835, 'CMCSA': 391, 'DIS': 81, 'WMG': 315, 'FOXA': 176, 'NFLX': 18, 'LYV': 56, 'DISCA': 197, 'PARA': 166, 'CURI': 915, 'ROKU': 20}
Funds remaining (HRP): $8.45


In [100]:
'mCVAR'

S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

{'CMCSA': 0.0, 'CURI': 0.0, 'DIS': 0.0, 'DISCA': 0.0, 'FOXA': 0.0, 'LYV': 0.0, 'NFLX': 0.0, 'PARA': 0.0, 'ROKU': 1.0, 'T': 0.0, 'WMG': 0.0}


In [101]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_cvar.greedy_portfolio()
print('Discrete allocation (CVAR):', allocation)
print('Funds remaining (CVAR): ${:.2f}'.format(leftover))

Discrete allocation (CVAR): {'ROKU': 716, 'T': 1, 'CURI': 1}
Funds remaining (CVAR): $11.72


C:\Users\Halis Karic\Ana\lib\site-packages\pypfopt\discrete_allocation.py:210: RuntimeWarning: invalid value encountered in true_divide
  current_weights /= current_weights.sum()
